In [3]:
# Script Name: Option_1 (Arbitrage Option1 Sequential - Async Real Trade Execution, Delta Neutral)
import ccxt
import time
import datetime
from zoneinfo import ZoneInfo
import threading
import hmac
import hashlib
import base64
import requests

# API Keys (Bybit and Bitget only for this trial)
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'
BITGET_API_PASSWORD = 'BgSecure789Trade2025'

# === Connect to Spot APIs ===
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True, 'defaultType': 'spot'}
})

bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'password': BITGET_API_PASSWORD,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True, 'defaultType': 'spot'}
})


In [5]:
# === Settings ===
SPREAD_THRESHOLD = 0.004
MIN_VOLUME = 50000
CHECK_INTERVAL = 2
TRADE_USD_SIZE = 10
LOCAL_TIMEZONE = ZoneInfo("Asia/Jakarta")
TRADING_FEES = {'Bybit': 0.001, 'Bitget': 0.0008}
exchanges = {'Bybit': bybit, 'Bitget': bitget}
futures_markets_cache = {}

# === Utility Functions ===
def ask_permission_to_trade(exchange_name, symbol, spread, net_profit):
    timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%H:%M:%S')
    prompt = (f"\n⏰ {timestamp} | ⚠️ Arbitrage Signal on {exchange_name} - {symbol}\n"
              f"Spread: {spread:.4%} | Net Profit: {net_profit:.4%}\n"
              f"Do you want to execute this trade for {exchange_name} {symbol}? (yes/no): ")
    choice = input(prompt).strip().lower()
    return choice == 'yes'

def get_best_bid_ask(exchange, symbol):
    order_book = exchange.fetch_order_book(symbol)
    best_bid = order_book['bids'][0][0] if order_book['bids'] else None
    best_ask = order_book['asks'][0][0] if order_book['asks'] else None
    return best_bid, best_ask

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    match = [f for f in futures_list if f.startswith(f"{base}/USDT") or f.startswith(f"{base}-USDT")]
    return match[0] if match else None

def get_price(exchange, symbol):
    try:
        return exchange.fetch_ticker(symbol).get("last", None)
    except Exception:
        return None

def set_bitget_hedge_mode(api_key, api_secret, api_passphrase):
    try:
        url = "https://api.bitget.com/api/mix/v1/account/setPositionMode"
        timestamp = str(int(time.time() * 1000))
        method = "POST"
        body = '{"marginCoin":"USDT","positionMode":"hedge_mode"}'
        pre_hash_string = timestamp + method + "/api/mix/v1/account/setPositionMode" + body
        sign = base64.b64encode(
            hmac.new(api_secret.encode(), pre_hash_string.encode(), hashlib.sha256).digest()
        ).decode()

        headers = {
            "ACCESS-KEY": api_key,
            "ACCESS-SIGN": sign,
            "ACCESS-TIMESTAMP": timestamp,
            "ACCESS-PASSPHRASE": api_passphrase,
            "Content-Type": "application/json"
        }

        res = requests.post(url, headers=headers, data=body)
        if res.status_code == 200:
            print("🔧 Bitget hedge mode set successfully.")
        else:
            print(f"❌ Bitget hedge mode set failed: {res.status_code} {res.text}")
    except Exception as e:
        print("❌ Exception setting Bitget hedge mode:", e)

# === Trade Execution Thread ===
def handle_trade_thread(exchange_name, symbol, spot_price, futures_price):
    try:
        exchange = exchanges[exchange_name]
        base_asset = symbol.split('/')[0]
        futures_list = futures_markets_cache.get(exchange_name, [])
        futures_symbol = find_futures_symbol(symbol, futures_list)

        total_amount = TRADE_USD_SIZE / spot_price
        best_bid, _ = get_best_bid_ask(exchange, symbol)
        if not best_bid:
            print("❌ Cannot determine best bid.")
            return

        print(f"\n🚀 Starting SPOT LIMIT BUY: {total_amount:.6f} {base_asset} @ {best_bid:.4f}")
        order = exchange.create_order(symbol=symbol, type='limit', side='buy', amount=total_amount, price=best_bid)

        cumulative_fill = 0
        remaining_amount = total_amount
        wait_time = 0
        max_wait = 300

        while wait_time < max_wait:
            current_order = exchange.fetch_order(order['id'], symbol)
            filled = current_order['filled']
            new_best_bid, _ = get_best_bid_ask(exchange, symbol)

            if filled > 0:
                cumulative_fill += filled
                print(f"🔄 Accumulated Fill: {cumulative_fill:.6f} / {total_amount:.6f}")
                remaining_amount = total_amount - cumulative_fill
                if remaining_amount <= 0:
                    break

            if current_order['status'] in ['open', 'partially_filled'] and new_best_bid and new_best_bid != current_order['price']:
                print(f"🔁 Price moved. Cancelling and retrying at {new_best_bid:.4f}")
                exchange.cancel_order(order['id'], symbol)
                order = exchange.create_order(symbol=symbol, type='limit', side='buy', amount=remaining_amount, price=new_best_bid)

            time.sleep(1)
            wait_time += 1

        if cumulative_fill == 0:
            print("⚠️ Spot limit order not filled. Switching to market buy.")
            market_order = exchange.create_order(symbol=symbol, type='market', side='buy', amount=None, params={'cost': TRADE_USD_SIZE})
            cumulative_fill = market_order['filled']
            print(f"✅ Market Buy Filled: {cumulative_fill:.6f} {base_asset}")

        if cumulative_fill == 0:
            print("❌ Market order failed. Aborting futures leg.")
            return

        if exchange_name == 'Bybit':
            futures_exchange = ccxt.bybit({
                'apiKey': BYBIT_API_KEY,
                'secret': BYBIT_API_SECRET,
                'enableRateLimit': True,
                'options': {'defaultType': 'future'}
            })
        elif exchange_name == 'Bitget':
            futures_exchange = ccxt.bitget({
                'apiKey': BITGET_API_KEY,
                'secret': BITGET_API_SECRET,
                'password': BITGET_API_PASSWORD,
                'enableRateLimit': True,
                'options': {'defaultType': 'swap'}
            })
            futures_exchange.options['createMarketBuyOrderRequiresPrice'] = False
            set_bitget_hedge_mode(BITGET_API_KEY, BITGET_API_SECRET, BITGET_API_PASSWORD)
        else:
            print(f"❌ No futures exchange configured for {exchange_name}")
            return

        futures_exchange.set_leverage(1, futures_symbol)
        order_futures = futures_exchange.create_order(symbol=futures_symbol, type='market', side='sell', amount=cumulative_fill)

        print(f"✅ Futures Order Placed: {order_futures['id']}")
        print(f"🏁 Trade Complete for {symbol} | Matched Amount: {cumulative_fill:.6f}")

    except Exception as e:
        print(f"❌ Trade Error: {e}")

# === Core Logic ===
def check_intra_exchange_arbitrage(exchange, exchange_name, symbol, spot_price, futures_price):
    trading_fee = TRADING_FEES.get(exchange_name, 0.001)
    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (trading_fee * 2)
    if net_profit >= SPREAD_THRESHOLD:
        timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S %Z')
        print(f"\n🔍 Arbitrage Opportunity on {exchange_name}: {symbol} | {timestamp}")
        print(f"    ✅ Spot Price: {spot_price:.5f} USDT")
        print(f"    ✅ Futures Price: {futures_price:.5f} USDT")
        print(f"    ✅ Spread: {spread:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
        if ask_permission_to_trade(exchange_name, symbol, spread, net_profit):
            threading.Thread(target=handle_trade_thread, args=(exchange_name, symbol, spot_price, futures_price)).start()

def process_tokens(exchange, exchange_name, tokens):
    futures_list = futures_markets_cache.get(exchange_name, [])
    for token in tokens:
        futures_symbol = find_futures_symbol(token, futures_list)
        if not futures_symbol:
            continue
        spot_price = get_price(exchange, token)
        futures_price = get_price(exchange, futures_symbol)
        if spot_price and futures_price:
            check_intra_exchange_arbitrage(exchange, exchange_name, token, spot_price, futures_price)

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable_tokens = []
        futures_markets_cache[exchange_name] = [symbol for symbol in markets if ":USDT" in symbol or "-USDT-SWAP" in symbol]
        for symbol, market in markets.items():
            if exchange_name == "Bitget" and not market.get('spot', False):
                continue
            if '/USDT' in symbol and ':USDT' not in symbol and '-USDT-SWAP' not in symbol:
                try:
                    market_data = exchange.fetch_ticker(symbol)
                    volume = market_data.get('quoteVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except Exception:
                    continue
        print(f"✅ Found {len(tradable_tokens)} tradable tokens on {exchange_name}")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens from {exchange_name}: {e}")
        return []

def main():
    while True:
        try:
            for exchange_name, exchange in exchanges.items():
                tradable_tokens = get_tradable_tokens(exchange, exchange_name)
                if not tradable_tokens:
                    print(f"❌ No tradable tokens found on {exchange_name}. Retrying in 10 seconds...")
                    time.sleep(10)
                    continue
                print(f"🔄 Checking for arbitrage opportunities on {exchange_name}...")
                process_tokens(exchange, exchange_name, tradable_tokens)
            print("✅ Cycle complete. Waiting for next check...")
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Error in main loop: {e}")
            time.sleep(10)

if __name__ == "__main__":
    main()


✅ Found 396 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
✅ Found 549 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...

🔍 Arbitrage Opportunity on Bitget: ZEREBRO/USDT | 2025-04-18 23:40:55 WIB
    ✅ Spot Price: 0.01980 USDT
    ✅ Futures Price: 0.01995 USDT
    ✅ Spread: 0.7576%
    ✅ Net Profit After Fees: 0.5976%



⏰ 23:40:55 | ⚠️ Arbitrage Signal on Bitget - ZEREBRO/USDT
Spread: 0.7576% | Net Profit: 0.5976%
Do you want to execute this trade for Bitget ZEREBRO/USDT? (yes/no):  yes



🚀 Starting SPOT LIMIT BUY: 505.050505 ZEREBRO @ 0.0199
✅ Cycle complete. Waiting for next check...
🔄 Accumulated Fill: 505.050000 / 505.050505
🔄 Accumulated Fill: 1010.100000 / 505.050505
❌ Bitget hedge mode set failed: 400 {"code":"400172","msg":"The holding mode cannot be empty","requestTime":1744994779260,"data":null}
❌ Trade Error: bitget {"code":"40774","msg":"The order type for unilateral position must also be the unilateral position type.","requestTime":1744994781270,"data":null}
✅ Found 396 tradable tokens on Bybit
🔄 Checking for arbitrage opportunities on Bybit...
✅ Found 550 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bitget...

🔍 Arbitrage Opportunity on Bitget: ZEREBRO/USDT | 2025-04-18 23:49:38 WIB
    ✅ Spot Price: 0.01960 USDT
    ✅ Futures Price: 0.01976 USDT
    ✅ Spread: 0.8163%
    ✅ Net Profit After Fees: 0.6563%



⏰ 23:49:38 | ⚠️ Arbitrage Signal on Bitget - ZEREBRO/USDT
Spread: 0.8163% | Net Profit: 0.6563%
Do you want to execute this trade for Bitget ZEREBRO/USDT? (yes/no):  no



KeyboardInterrupt

